In [3]:
import pygame as pg
import random


pg.init()

WHITE = (255, 255, 255)
GRAY = (190, 200, 220)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Set the size of each cell on the game board
CELL_SIZE = 130

# Set the size of the game window
WINDOW_SIZE = (CELL_SIZE*4, CELL_SIZE*4)
screen = pg.display.set_mode(WINDOW_SIZE)

# Set the caption for the game window
pg.display.set_caption("Sudoku")

# Create a 4x4 grid of cells with initial values
grid = [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]
# Define the initial grid values
matrix = [[0 for i in range(4)] for j in range(4)]

def is_valid(matrix, row, col, value):
    if value in matrix[row]:
        return False
    if value in [matrix[i][col] for i in range(4)]:
        return False
    
    sub_box_row = (row // 2) * 2
    sub_box_col = (col // 2) * 2
    sub_box_values = [matrix[i][j] for i in range(sub_box_row, sub_box_row + 2)
                                       for j in range(sub_box_col, sub_box_col + 2)]
    if value in sub_box_values:
        return False
    return True

# Define function to generate a Sudoku puzzle
def sudoku_gen(matrix):
    for row in range(4):
        for col in range(4):       
            valid_values = [i for i in range(1, 5) if is_valid(matrix, row, col, i)]
            if valid_values:
                matrix[row][col] = random.choice(valid_values)
            else:
                return sudoku_gen(matrix)

    return matrix
def hide_cells(matrix):
    num_to_hide = random.randint(1,8)
    cells_to_hide = random.sample([(i, j) for i in range(4) for j in range(4) if grid[i][j] != 0], num_to_hide)
    for row, col in cells_to_hide:
        grid[row][col] = 0


matri = sudoku_gen(grid)
hide_cells(matri)
def is_valid(grid, row, col):   
    # Check row and column for duplicates
    for i in range(4):
        if i != col and grid[row][i] == grid[row][col]:
            return False
        if i != row and grid[i][col] == grid[row][col]:
            return False

    # Check 2x2 sub-grid for duplicates
    sub_grid_row = (row // 2) * 2
    sub_grid_col = (col // 2) * 2
    for i in range(sub_grid_row, sub_grid_row + 2):
        for j in range(sub_grid_col, sub_grid_col + 2):
            if (i != row or j != col) and grid[i][j] == grid[row][col]:
                return False

    return True

# Main game loop
running = True
selected_cell = None
while running:
    # Handle events
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
        elif event.type == pg.MOUSEBUTTONUP:
            pos = pg.mouse.get_pos()
            row = pos[1] // CELL_SIZE
            col = pos[0] // CELL_SIZE
            selected_cell = (row, col)
        elif event.type == pg.KEYDOWN:
            if selected_cell is not None:
                row, col = selected_cell
                if event.unicode.isdigit() and int(event.unicode) != 0:
                    grid[row][col] = int(event.unicode)
                elif event.key == pg.K_BACKSPACE:
                    grid[row][col] = 0
                    selected_cell = None

    # Clear the screen
    screen.fill(GRAY)

    # Draw the grid
    for row in range(4):
        for col in range(4):
            rect = pg.Rect(col*CELL_SIZE, row*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            pg.draw.rect(screen, BLACK, rect, 1)
            font = pg.font.SysFont(None, 48)
            if grid[row][col] != 0:
                text = font.render(str(grid[row][col]), True, BLACK)
                text_rect = text.get_rect(center=rect.center)
                screen.blit(text, text_rect)

            # Highlight the selected cell in green
            if selected_cell is not None and (row, col) == selected_cell:
                pg.draw.rect(screen, GREEN, rect, 2)

            # Highlight incorrect cells in red
            if grid[row][col] != 0 and not is_valid(grid, row, col):
                pg.draw.rect(screen, RED, rect, 2)
            # Check if all cells are filled
            if all(grid[row][col] != 0 for row in range(4) for col in range(4)):
                font = pg.font.SysFont(None, 72)
                text = font.render("YOU WIN!", True, GREEN)
                text_rect = text.get_rect(center=(WINDOW_SIZE[0]//2, WINDOW_SIZE[1]//2))
                screen.blit(text, text_rect)

    # Update the display
    pg.display.update()

# Quit Pygame
pg.quit()
